In [22]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process


[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

point_projections = generate_projections()
stat_projections = generate_stat_projections()

projections_data = append_stat_projections(point_projections, stat_projections, 4)

### Player Removal

In [24]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


# 2024/25 GW3 Challenge: Too Big at the Back - Clean sheets are worth double points. Your squad may only contain 2 Goalkeepers and 3 Defenders.

### Points Refinement

In [25]:
# Keep only Defender and Goalkeeper positions
projections_data = projections_data[(projections_data['Position'] == 'Defender') | (projections_data['Position'] == 'Goalkeeper')]
# Reset index
projections_data = projections_data.reset_index(drop=True)

def adjust_clean_sheet_points(df):
    additional_cs_points = 4 * df['Clean_Sheet']
    new_xpts = df['Predicted_Points'] + additional_cs_points
    df['Predicted_Points'] = new_xpts

display(projections_data)
adjust_clean_sheet_points(projections_data)
display(projections_data)

,ID,Name,Team,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
0,3,Gabriel,Arsenal,Defender,6.0,4.23,90,Spurs (A),0.121,0.073,0.185,0.250,0.137
1,6,J.Timber,Arsenal,Defender,5.5,2.12,59,Spurs (A),0.056,0.056,0.108,0.250,0.064
2,14,Ramsdale,Southampton,Goalkeeper,4.4,3.59,90,Man Utd (H),0.000,0.000,0.000,0.156,0.000
3,15,Raya,Arsenal,Goalkeeper,5.5,4.04,90,Spurs (A),0.000,0.000,0.000,0.250,0.000
4,18,Saliba,Arsenal,Defender,6.0,3.57,90,Spurs (A),0.056,0.036,0.090,0.250,0.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,641,Zych,Aston Villa,Goalkeeper,4.0,0.00,0,Everton (H),0.000,0.000,0.000,0.000,0.000
151,643,Casey,West Ham,Defender,4.0,0.00,0,Fulham (A),0.000,0.000,0.000,0.000,0.000
152,649,Meghoma,Brentford,Defender,4.0,0.00,0,Man City (A),0.000,0.000,0.000,0.000,0.000
153,650,Lacroix,Crystal Palace,Defender,4.5,3.55,79,Leicester (H),0.000,0.000,0.000,0.000,0.000


,ID,Name,Team,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
0,3,Gabriel,Arsenal,Defender,6.0,5.230,90,Spurs (A),0.121,0.073,0.185,0.250,0.137
1,6,J.Timber,Arsenal,Defender,5.5,3.120,59,Spurs (A),0.056,0.056,0.108,0.250,0.064
2,14,Ramsdale,Southampton,Goalkeeper,4.4,4.214,90,Man Utd (H),0.000,0.000,0.000,0.156,0.000
3,15,Raya,Arsenal,Goalkeeper,5.5,5.040,90,Spurs (A),0.000,0.000,0.000,0.250,0.000
4,18,Saliba,Arsenal,Defender,6.0,4.570,90,Spurs (A),0.056,0.036,0.090,0.250,0.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,641,Zych,Aston Villa,Goalkeeper,4.0,0.000,0,Everton (H),0.000,0.000,0.000,0.000,0.000
151,643,Casey,West Ham,Defender,4.0,0.000,0,Fulham (A),0.000,0.000,0.000,0.000,0.000
152,649,Meghoma,Brentford,Defender,4.0,0.000,0,Man City (A),0.000,0.000,0.000,0.000,0.000
153,650,Lacroix,Crystal Palace,Defender,4.5,3.550,79,Leicester (H),0.000,0.000,0.000,0.000,0.000


### Optimisation

In [26]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw3-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5 (2 goalkeepers + 3 defenders)
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 2 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 2

# Exactly 3 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) == 3

# Budget constraint: Total cost must be less than or equal to 30m
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 30

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal
